In [ ]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from IPython.display import HTML, display
import ipywidgets as widgets
import os
import re
import getpass

In [ ]:
SYS_NAMES = ["prelude", "prelude-upgrade6", "clone", "dev"] 
connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)
user_name = getpass.getuser()

In [ ]:
sql_query = "SELECT * FROM {0};"

df = pd.read_sql(sql_query.format('ELN_WRITEUP_SCRAPPED'), engine)
# df.head()

In [ ]:
chem_colour = "#004466"   # Blue

def parse_table(html_table):
    """Parse table headers values into a list."""
    soup = BeautifulSoup(html_table, 'html.parser')
    values = [cell.get_text(strip=True) for cell in soup.select('tbody td') if len(cell.get_text(strip=True)) >= 2]
    return values

def color_code_writeup(row):
    """Color-code reactants, solvents, and products in the write-up column."""
    write_up = remove_styles(row['write_up'])
    
    reactants = parse_table(row['reactants_table'])
    solvents = parse_table(row['solvents_table'])
    products = parse_table(row['products_table'])
    
    chem_agents = reactants + solvents + products

    if chem_agents:
        pattern = r'(' + '|'.join(re.escape(value) for value in chem_agents if value) + r')'
        write_up = re.sub(pattern, rf'<span style="color: {chem_colour};">\1</span>', write_up, flags=re.IGNORECASE)
    
    return write_up

In [ ]:
def remove_styles(html_output):
    soup = BeautifulSoup(html_output, 'html.parser')
    for tag in soup.find_all(style=True):
        del tag['style']
    return soup.prettify()
    

dir_path = "./exp_ids"
unique_suffixes = set()

for filename in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, filename)):
        if 'prod_' in filename:
            suffix = filename.split('prod_')[-1]
        elif 'up6_' in filename:
            suffix = filename.split('up6_')[-1]
        else:
            continue
        unique_suffixes.add(suffix)

cro_prots = [s.split('.')[0] for s in unique_suffixes]
# print(cro_prots)

In [ ]:
central_css = """
<style>
/* Container for each experiment */
.experiment-container {
    border: 1px solid #ccc;
    margin: 10px 0;
    padding: 10px;
    background-color: #f9f9f9;
}

/* Header section for experiment ID and date */
.experiment-header {
    margin-bottom: 10px;
}

/* Flexbox container for columns */
.column-container {
    display: flex;
    flex-wrap: nowrap;
    gap: 10px;
}

/* Common styles for each column */
.column {
    flex: 1;
    border: 1px solid #ddd;
    padding: 10px;
}

/* Horizontal scrolling for the tables column */
.tables-column {
    overflow-x: auto;
    white-space: nowrap;
}

/* Vertical scrolling for write-up columns */
.writeup-column {
    overflow-y: auto;
    max-height: 300px; /* Optional: Limit height for scrolling */
}

/* Hidden content toggle visibility */
.hidden {
    display: none;
}

.visible {
    display: block;
}

/* Spacing for toggled content */
.toggled-content {
    margin-top: 10px;
}

/* Styling for table header */
table thead {
    font-weight: bold;
}

/* Styling for table rows */
td {
    border: 1px solid #ccc;
    margin: 10px 0;
    padding: 10px;
    background-color: #f9f9f9;
}

/* Custom styles for checkboxes */
.checkbox-container {
    display: flex;
    align-items: center;
}
.checkbox-container label {
    margin-right: 15px;
}
</style>
"""

In [ ]:
def render_html_group(idx, group):
    exp_id = group['exp_id'].iloc[0]
    created_date = group['created_date'].iloc[0]

    write_up_prod = group.loc[group['system_name'] == SYS_NAMES[0], 'write_up'].values[0] if SYS_NAMES[0] in group['system_name'].values else '<div>non-existent write-up</div>'
    write_up_up6 = group.loc[group['system_name'] == SYS_NAMES[1], 'write_up'].values[0] if SYS_NAMES[1] in group['system_name'].values else '<div>non-existent write-up</div>'
    # write_up_clone = group.loc[group['system_name'] == SYS_NAMES[2], 'write_up'].values[0] if SYS_NAMES[2] in group['system_name'].values else '<div>non-existent write-up</div>'

    return f"""
    {central_css}
    <div class="experiment-container">
    <div class="experiment-header">
        <strong>Row: </strong> {idx} <br>
        <strong>Experiment ID:</strong> {exp_id} <br>
        <strong>Created Date:</strong> {created_date}
    </div>
    <div class="column-container">
        <div class="column tables-column">
            <h3>Chemical Reagents Table</h3>
            <button onclick="toggleVisibility('reactants-{exp_id}')">Reactants</button>
            <div id="reactants-{exp_id}" class="hidden toggled-content">{remove_styles(group['reactants_table'].iloc[0])}</div>

            <button onclick="toggleVisibility('solvents-{exp_id}')">Solvents</button>
            <div id="solvents-{exp_id}" class="hidden toggled-content">{remove_styles(group['solvents_table'].iloc[0])}</div>

            <button onclick="toggleVisibility('products-{exp_id}')">Products</button>
            <div id="products-{exp_id}" class="hidden toggled-content">{remove_styles(group['products_table'].iloc[0])}</div>
        </div>
        <div class="column writeup-column">
            <h3>Production Write-Up</h3>
            {write_up_prod}
        </div>
        <div class="column writeup-column">
            <h3>Upgrade6 Write-Up</h3>
            {write_up_up6}
        </div>
        </div>
        <div class="checkbox-container">
            <label><input type="checkbox" id="valid-{exp_id}" class="valid-checkbox"> Valid</label>
            <label><input type="checkbox" id="invalid-{exp_id}" class="invalid-checkbox"> Invalid</label>
        </div>
    </div>
"""

In [ ]:
file_name_prefix = 'exp_ids_eln_writeup_prod_{0}.txt'
js_code = """
<script>
    function toggleVisibility(id) {
        const element = document.getElementById(id);
        if (element.style.display === "none") {
            element.style.display = "block";
        } else {
            element.style.display = "none";
        }
    }

    window.rowStates = {};
    
    function updateRowState(expId, validChecked, invalidChecked) {
        window.rowStates[expId] = {
            VALID: validChecked ? 1 : 0,
            INVALID: invalidChecked ? 1 : 0
        };
    }

    document.querySelectorAll('.valid-checkbox').forEach((checkbox) => {
        checkbox.addEventListener('change', (event) => {
            const expId = event.target.id.split('-')[1];
            
            if (event.target.checked) {
                document.getElementById('invalid-' + expId).checked = false;
            }
            
            updateRowState(expId, event.target.checked, document.getElementById('invalid-' + expId).checked);
        });
    });

    document.querySelectorAll('.invalid-checkbox').forEach((checkbox) => {
        checkbox.addEventListener('change', (event) => {
            const expId = event.target.id.split('-')[1];
            
            if (event.target.checked) {
                document.getElementById('valid-' + expId).checked = false;
            }
            
            updateRowState(expId, document.getElementById('valid-' + expId).checked, event.target.checked);
        });
    });

"""

In [ ]:
js_fx = f"""
    function postValidityBackend() {{
        fetch('http://localhost:5000/api/update-data?user_name={user_name.replace('-', '_')}', {{
          method: 'POST',
          headers: {{
            'Content-Type': 'application/json'
          }},
          body: JSON.stringify(window.rowStates)
        }})
        .then(response => response.json())
        .then(data => {{
          if (data.status === 'success') {{
            console.log("Validity updated successfully!");
          }} else {{
            console.error("Error updating row states: ", data.message);
          }}
        }})
        .catch(error => {{
          console.error('Error:', error);
        }});
    }}
    
</script>
"""

js_code = f"""
    {js_code}
    {js_fx}
"""